In [12]:
import pandas as pd
import numpy as np


In [13]:
data=pd.read_csv("../data/Silver/annotation/Other_Label.xlsx - Sheet1.csv")

In [14]:
# Đổi tên cột
data.rename(columns={"content":"reviews","title":"sentiment","aspect":"aspect","type":"type_product"},inplace=True)
data=data.dropna()




In [15]:
data

,sentiment,reviews,type_product,aspect,assess,aspect1,assess1,aspect2,assess2
0,5,Cảm thấy tiki gói hàng okela hơn đó nha:D,0,4,5.0,4,4.0,4,4.0
1,5,Sách mới uy tín chất lượng dịch vụ tốt...,0,2,5.0,2,5.0,2,5.0
2,5,"mình vừa nhận được sách , sách được bao bì cẩn...",0,4,5.0,4,4.0,4,4.0
3,5,"Mua nhầm, sách PDF free r mn ạ",0,0,1.0,0,3.0,0,3.0
4,5,Sách dính keo ở mỗi trang nên khi mình lật qua...,0,4,3.0,4,1.0,4,1.0
...,...,...,...,...,...,...,...,...,...
1499,5,sản phẩm xài ok,25,2,4.0,2,4.0,2,4.0
1500,5,An tâm,25,2,5.0,2,5.0,2,5.0
1501,4,"Khá sạch, mùi dễ chịu",25,2,5.0,2,5.0,2,5.0
1502,4,bọt ít k sạch dầu mỡ,25,2,2.0,2,2.0,2,2.0


In [16]:
data.rename(columns={"assess1":"sentiment_handle1","assess2":"sentiment_handle2","assess":"sentiment_handle3","aspect1":"aspect_handle1","aspect2":"aspect_handle2","aspect":"aspect_handle3"},inplace=True)


In [17]:
data = data[data['aspect_handle3'] != '0']

data["aspect_handle2"].unique()

array(['4', '2', '1,2,3,4', '2,4', '0,2', '2,3,4', '1,2', '2,3', '1',
       '1,2,3', '1,4', '0,4', '3', '1,2,4'], dtype=object)

In [18]:
# Hàm chuyển đổi chuỗi dạng "1,2,3,4" thành danh sách [1,2,3,4]

def convert_to_list(value):
    if pd.isna(value) or value == '':
        return []
    try:
        # Tách chuỗi theo dấu phẩy và chuyển thành số nguyên
        return [int(x.strip()) for x in value.split(',')]
    except:
        return [int(value)] if value.strip().isdigit() else []

# Chuyển đổi các cột aspect_handle thành danh sách
data['aspect_handle1'] = data['aspect_handle1'].astype(str).apply(convert_to_list)
data['aspect_handle2'] = data['aspect_handle2'].astype(str).apply(convert_to_list)
data['aspect_handle3'] = data['aspect_handle3'].astype(str).apply(convert_to_list)

# Kiểm tra kết quả sau khi chuyển đổi
print("Kiểu dữ liệu và giá trị mẫu sau khi chuyển đổi:")
print(f"aspect_handle1: {type(data['aspect_handle1'].iloc[0])}, Mẫu: {data['aspect_handle1'].iloc[0:3].tolist()}")
print(f"aspect_handle2: {type(data['aspect_handle2'].iloc[0])}, Mẫu: {data['aspect_handle2'].iloc[0:3].tolist()}")
print(f"aspect_handle3: {type(data['aspect_handle3'].iloc[0])}, Mẫu: {data['aspect_handle3'].iloc[0:3].tolist()}")


Kiểu dữ liệu và giá trị mẫu sau khi chuyển đổi:
aspect_handle1: <class 'list'>, Mẫu: [[4], [2], [4]]
aspect_handle2: <class 'list'>, Mẫu: [[4], [2], [4]]
aspect_handle3: <class 'list'>, Mẫu: [[4], [2], [4]]


C:\Users\phamd\AppData\Local\Temp\ipykernel_19748\2835673261.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['aspect_handle1'] = data['aspect_handle1'].astype(str).apply(convert_to_list)
C:\Users\phamd\AppData\Local\Temp\ipykernel_19748\2835673261.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['aspect_handle2'] = data['aspect_handle2'].astype(str).apply(convert_to_list)
C:\Users\phamd\AppData\Local\Temp\ipykernel_19748\2835673261.py:15: SettingWithCopyWarning: 
A value is trying to be s

TÍnh độ đồng thuận Kohen Kappa

In [19]:
data.isna().sum()

sentiment            0
reviews              0
type_product         0
aspect_handle3       0
sentiment_handle3    0
aspect_handle1       0
sentiment_handle1    0
aspect_handle2       0
sentiment_handle2    0
dtype: int64

In [20]:
from sklearn.metrics import cohen_kappa_score
import numpy as np

# Hàm để tính toán Cohen's Kappa cho các cặp đánh giá
def calculate_kappa_for_aspects(data):
    # Tạo các cặp đánh giá
    pairs = [
        ('aspect_handle1', 'aspect_handle2'),
        ('aspect_handle1', 'aspect_handle3'),
        ('aspect_handle2', 'aspect_handle3')
    ]
    
    results = {}
    
    for pair in pairs:
        col1, col2 = pair
        
        # Chuyển đổi danh sách thành chuỗi để so sánh
        aspect1_str = data[col1].apply(lambda x: ','.join(map(str, x)) if x else '')
        aspect2_str = data[col2].apply(lambda x: ','.join(map(str, x)) if x else '')
        
        # Loại bỏ các hàng có giá trị rỗng
        valid_indices = (aspect1_str != '') & (aspect2_str != '')
        
        if valid_indices.sum() > 0:
            # Tính Cohen's Kappa
            kappa = cohen_kappa_score(
                aspect1_str[valid_indices],
                aspect2_str[valid_indices]
            )
            results[f"{col1} vs {col2}"] = kappa
        else:
            results[f"{col1} vs {col2}"] = np.nan
    
    return results

# Tính toán độ đồng thuận cho các cột aspect
aspect_kappa = calculate_kappa_for_aspects(data)
print("Độ đồng thuận Cohen's Kappa cho các cột aspect:")
for pair, score in aspect_kappa.items():
    print(f"{pair}: {score:.4f}")

# Hàm để tính toán Cohen's Kappa cho các cặp đánh giá sentiment
def calculate_kappa_for_sentiments(data):
    # Tạo các cặp đánh giá
    pairs = [
        ('sentiment_handle1', 'sentiment_handle2'),
        ('sentiment_handle1', 'sentiment_handle3'),
        ('sentiment_handle2', 'sentiment_handle3')
    ]
    
    results = {}
    
    for pair in pairs:
        col1, col2 = pair
        
        # Loại bỏ các hàng có giá trị NaN
        valid_indices = data[col1].notna() & data[col2].notna()
        
        if valid_indices.sum() > 0:
            # Tính Cohen's Kappa
            kappa = cohen_kappa_score(
                data[col1][valid_indices],
                data[col2][valid_indices]
            )
            results[f"{col1} vs {col2}"] = kappa
        else:
            results[f"{col1} vs {col2}"] = np.nan
    
    return results

# Tính toán độ đồng thuận cho các cột sentiment
sentiment_kappa = calculate_kappa_for_sentiments(data)
print("\nĐộ đồng thuận Cohen's Kappa cho các cột sentiment:")
for pair, score in sentiment_kappa.items():
    print(f"{pair}: {score:.4f}")

# Tính trung bình độ đồng thuận
avg_aspect_kappa = np.mean(list(aspect_kappa.values()))
avg_sentiment_kappa = np.mean(list(sentiment_kappa.values()))

print(f"\nĐộ đồng thuận trung bình cho aspect: {avg_aspect_kappa:.4f}")
print(f"Độ đồng thuận trung bình cho sentiment: {avg_sentiment_kappa:.4f}")


Độ đồng thuận Cohen's Kappa cho các cột aspect:
aspect_handle1 vs aspect_handle2: 0.9984
aspect_handle1 vs aspect_handle3: 0.9938
aspect_handle2 vs aspect_handle3: 0.9953

Độ đồng thuận Cohen's Kappa cho các cột sentiment:
sentiment_handle1 vs sentiment_handle2: 0.9975
sentiment_handle1 vs sentiment_handle3: 0.7595
sentiment_handle2 vs sentiment_handle3: 0.7606

Độ đồng thuận trung bình cho aspect: 0.9958
Độ đồng thuận trung bình cho sentiment: 0.8392
